<a href="https://colab.research.google.com/github/overfit-ir/persian-twitter-ner/blob/master/fine-tune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [1]:
import pandas as pd
import numpy as np

In [2]:
!wget -q --show-progress 'https://raw.githubusercontent.com/overfit-ir/persian-twitter-ner/master/twitter_data/persian-ner-twitter-data/train.txt'
!wget -q --show-progress 'https://raw.githubusercontent.com/overfit-ir/persian-twitter-ner/master/twitter_data/persian-ner-twitter-data/test.txt'
!wget -q --show-progress 'https://raw.githubusercontent.com/overfit-ir/persian-twitter-ner/master/twitter_data/persian-ner-twitter-data/dev.txt'
!mkdir data && mv train.txt data && mv test.txt data && mv dev.txt data

train.txt           100%[===================>]   2.20M  --.-KB/s    in 0.1s    
test.txt            100%[===================>] 108.10K  --.-KB/s    in 0.02s   
dev.txt             100%[===================>] 160.83K  --.-KB/s    in 0.03s   


# Benchmark2

In [3]:
!cat data/train.txt | grep -v "^#" | cut -f 1,2 | tr '\t' ' ' > train.txt.tmp
!cat data/test.txt | grep -v "^#" | cut -f 1,2 | tr '\t' ' ' > test.txt.tmp
!cat data/dev.txt | grep -v "^#" | cut -f 1,2 | tr '\t' ' ' > dev.txt.tmp

In [4]:
!git clone https://github.com/huggingface/transformers

Cloning into 'transformers'...
remote: Enumerating objects: 24, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (21/21), done.
remote: Total 62634 (delta 6), reused 9 (delta 0), pack-reused 62610
Receiving objects: 100% (62634/62634), 47.73 MiB | 13.69 MiB/s, done.
Resolving deltas: 100% (44354/44354), done.


In [5]:
!pip -q install transformers/
!pip -q install sentencepiece

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 3.2MB 4.1MB/s 
     |████████████████████████████████| 890kB 33.7MB/s 
     |████████████████████████████████| 1.2MB 4.0MB/s 


In [7]:
!wget "https://raw.githubusercontent.com/stefan-it/fine-tuned-berts-seq/master/scripts/preprocess.py"

--2021-02-17 11:25:25--  https://raw.githubusercontent.com/stefan-it/fine-tuned-berts-seq/master/scripts/preprocess.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 991 [text/plain]
Saving to: ‘preprocess.py.1’

preprocess.py.1     100%[===================>]     991  --.-KB/s    in 0s      

2021-02-17 11:25:25 (41.7 MB/s) - ‘preprocess.py.1’ saved [991/991]



In [16]:
!python3 preprocess.py dev.txt.tmp HooshvareLab/bert-base-parsbert-uncased 256 > dev.txt
!python3 preprocess.py train.txt.tmp HooshvareLab/bert-base-parsbert-uncased 256 > train.txt
!python3 preprocess.py test.txt.tmp HooshvareLab/bert-base-parsbert-uncased 256 > test.txt

In [9]:
!pip -q install -r transformers/examples/token-classification/requirements.txt

     |████████████████████████████████| 51kB 2.6MB/s 
     |████████████████████████████████| 184kB 5.7MB/s 
     |████████████████████████████████| 102kB 4.9MB/s 
     |████████████████████████████████| 245kB 7.5MB/s 
     |████████████████████████████████| 20.7MB 1.4MB/s 


In [10]:
!pip -q install conllu

In [17]:
!mkdir processed_data
!mv train.txt processed_data/ && mv test.txt processed_data/ && mv dev.txt processed_data/

In [12]:
!cat processed_data/train.txt processed_data/test.txt processed_data/dev.txt | cut -d " " -f 2 | grep -v "^$"| sort | uniq > labels.txt

In [21]:
! cp /content/transformers/examples/legacy/token-classification/utils_ner.py /content/transformers/examples/token-classification
! cp /content/transformers/examples/legacy/token-classification/tasks.py /content/transformers/examples/token-classification

In [24]:
!python3 transformers/examples/token-classification/run_ner.py --data_dir ./processed_data/ \
--labels labels.txt \
--model_name_or_path HooshvareLab/bert-base-parsbert-uncased \
--output_dir eval/ \
--max_seq_length  256 \
--per_device_train_batch_size 32 \
--save_steps 750 \
--seed 1 \
--do_train \
--do_eval \
--do_predict

2021-02-17 11:43:39.893643: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
Traceback (most recent call last):
  File "transformers/examples/token-classification/run_ner.py", line 455, in <module>
    main()
  File "transformers/examples/token-classification/run_ner.py", line 156, in main
    model_args, data_args, training_args = parser.parse_args_into_dataclasses()
  File "/usr/local/lib/python3.6/dist-packages/transformers/hf_argparser.py", line 180, in parse_args_into_dataclasses
    obj = dtype(**inputs)
  File "<string>", line 13, in __init__
  File "transformers/examples/token-classification/run_ner.py", line 134, in __post_init__
    raise ValueError("Need either a dataset name or a training/validation file.")
ValueError: Need either a dataset name or a training/validation file.


In [ ]:
!zip -r eval.zip eval

  adding: eval/ (stored 0%)
  adding: eval/checkpoint-2250/ (stored 0%)
  adding: eval/checkpoint-2250/scheduler.pt (deflated 49%)
  adding: eval/checkpoint-2250/config.json (deflated 54%)
  adding: eval/checkpoint-2250/pytorch_model.bin (deflated 8%)
  adding: eval/checkpoint-2250/optimizer.pt (deflated 41%)
  adding: eval/checkpoint-2250/trainer_state.json (deflated 60%)
  adding: eval/checkpoint-2250/training_args.bin (deflated 45%)
  adding: eval/test_predictions.txt (deflated 71%)
  adding: eval/test_results.txt (deflated 34%)
  adding: eval/config.json (deflated 54%)
  adding: eval/checkpoint-750/ (stored 0%)
  adding: eval/checkpoint-750/scheduler.pt (deflated 49%)
  adding: eval/checkpoint-750/config.json (deflated 54%)
  adding: eval/checkpoint-750/pytorch_model.bin (deflated 8%)
  adding: eval/checkpoint-750/optimizer.pt (deflated 41%)
  adding: eval/checkpoint-750/trainer_state.json (deflated 45%)
  adding: eval/checkpoint-750/training_args.bin (deflated 45%)
  adding: eval/

In [ ]:
!zip -r runs.zip runs

  adding: runs/ (stored 0%)
  adding: runs/Jan28_05-40-21_c389095872f4/ (stored 0%)
  adding: runs/Jan28_05-40-21_c389095872f4/events.out.tfevents.1611840450.c389095872f4.263.2 (deflated 29%)
  adding: runs/Jan28_05-40-21_c389095872f4/events.out.tfevents.1611812466.c389095872f4.263.0 (deflated 54%)
  adding: runs/Jan28_05-40-21_c389095872f4/1611812466.8290157/ (stored 0%)
  adding: runs/Jan28_05-40-21_c389095872f4/1611812466.8290157/events.out.tfevents.1611812466.c389095872f4.263.1 (deflated 59%)


In [ ]:
!ls -lh

total 4.4G
drwxr-xr-x  2 root root 4.0K Jan 28 05:38 data
-rw-r--r--  1 root root 161K Jan 28 05:38 dev.txt.tmp
drwxr-xr-x  5 root root 4.0K Jan 28 13:29 eval
-rw-r--r--  1 root root 4.4G Jan 28 13:49 eval.zip
-rw-r--r--  1 root root   74 Jan 28 05:39 labels.txt
-rw-r--r--  1 root root  991 Jan 28 05:39 preprocess.py
drwxr-xr-x  2 root root 4.0K Jan 28 13:27 processed_data
drwxr-xr-x  3 root root 4.0K Jan 28 05:41 runs
-rw-r--r--  1 root root 4.8K Jan 28 13:49 runs.zip
drwxr-xr-x  1 root root 4.0K Jan 20 17:27 sample_data
-rw-r--r--  1 root root 109K Jan 28 05:38 test.txt.tmp
-rw-r--r--  1 root root 2.3M Jan 28 05:38 train.txt.tmp
drwxr-xr-x 16 root root 4.0K Jan 28 05:38 transformers
